In [94]:
import pandas as pd

# Load both sheets FIRST
df_vehicles = pd.read_excel("Traffic_updated.xlsx", sheet_name='Vehicles', engine='openpyxl')
df_drivers = pd.read_excel("Traffic_updated.xlsx", sheet_name='Drivers', engine='openpyxl')
df_speed_violations = pd.read_excel("Traffic_updated.xlsx", sheet_name='SpeedViolations', engine='openpyxl')


df.head()

,VehicleID,PlateNumber,VehicleType,OwnerID,PlateNumber_Cleaned,VehicleType_Cleaned,VehicleType_Encoded
0,1,92X 5338,Car,64,92X5338,Car,1
1,2,186-ZJJ,Car,145,186ZJJ,Car,1
2,3,835 1WK,Car,140,8351WK,Car,1
3,4,OYD7711,Bus,70,OYD7711,Bus,2
4,5,KFU-0955,Bus,24,KFU0955,Bus,2


In [226]:
df.tail()

,VehicleID,PlateNumber,VehicleType,OwnerID,PlateNumber_Cleaned,VehicleType_Cleaned,VehicleType_Encoded
195,196,9MR 974,Car,112,9MR974,Car,1
196,197,AI-9234,Bus,60,AI9234,Bus,2
197,198,6538,Motorcycle,72,6538,Motorcycle,4
198,199,XPA6114,Motorcycle,128,XPA6114,Motorcycle,4
199,200,14K X29,Car,39,14KX29,Car,1


## DIMENSION TABLE CLEANING

##### 1. VehicleID

In [98]:
#1) Missing values
missing_vehicle_ids = df['VehicleID'].isnull().sum()
print(f"Missing VehicleID values: {missing_vehicle_ids}")

#2) Duplicates
duplicate_vehicle_ids = df['VehicleID'].duplicated().sum()
print(f"Duplicate VehicleID values: {duplicate_vehicle_ids}")

#3) If data type is integer & convert if not:
if df['VehicleID'].dtype != 'int64':
    df['VehicleID'] = pd.to_numeric(df['VehicleID'], errors='coerce')
    print("Converted VehicleID to integer.")

#4) Droping rows w/ missing or duplicate values (in case)
df_cleaned = df.dropna(subset=['VehicleID'])
df_cleaned = df_cleaned[~df_cleaned['VehicleID'].duplicated()]

#5) Final confirmation
print("Final check — are VehicleIDs unique?:", df_cleaned['VehicleID'].is_unique)

Missing VehicleID values: 0
Duplicate VehicleID values: 0
Final check — are VehicleIDs unique?: True


##### 2. Plate number

In [101]:
#1) Checking for missing values
missing_plates = df['PlateNumber'].isnull().sum()
print(f"Missing PlateNumber values: {missing_plates}")

#2) Checking if clean & standardize PlateNumber
def clean_plate(plate):
    if pd.isnull(plate):
        return "UNKNOWN"
    cleaned = re.sub(r'[^A-Za-z0-9]', '', str(plate))  # Remove special characters
    return cleaned.upper()  # Standardize to uppercase

#3) Cleaning function
df['PlateNumber_Cleaned'] = df['PlateNumber'].apply(clean_plate)

#4) See some before/after examples for sample
print(df[['PlateNumber', 'PlateNumber_Cleaned']].head(10))


Missing PlateNumber values: 0
  PlateNumber PlateNumber_Cleaned
0    92X 5338             92X5338
1     186-ZJJ              186ZJJ
2     835 1WK              8351WK
3     OYD7711             OYD7711
4    KFU-0955             KFU0955
5     41T•740              41T740
6     93P L10              93PL10
7    0N H8535             0NH8535
8     Q81 4GN              Q814GN
9     049-PVL              049PVL


##### 3. Vehicle type

In [104]:
#1) Checking unique values before cleaning
print("Original VehicleType values:")
print(df['VehicleType'].value_counts(dropna=False))

#2) Cleaning and standardize the categories
def clean_vehicle_type(vtype):
    if pd.isnull(vtype):
        return "Unknown"
    vtype = str(vtype).strip().lower()
    if "car" in vtype:
        return "Car"
    elif "bus" in vtype:
        return "Bus"
    elif "truck" in vtype:
        return "Truck"
    elif "motor" in vtype or "bike" in vtype:
        return "Motorcycle"
    else:
        return "Other"

df['VehicleType_Cleaned'] = df['VehicleType'].apply(clean_vehicle_type)

#3) Encoding categories to integers
vehicle_type_encoding = {
    "Car": 1,
    "Bus": 2,
    "Truck": 3,
    "Motorcycle": 4,
    "Other": 5,
    "Unknown": 0
}
df['VehicleType_Encoded'] = df['VehicleType_Cleaned'].map(vehicle_type_encoding)

#4) Check cleaned distribution
print("\nCleaned VehicleType values:")
print(df['VehicleType_Cleaned'].value_counts())

#5) Preview results
print(df[['VehicleType', 'VehicleType_Cleaned', 'VehicleType_Encoded']].head(10))

Original VehicleType values:
VehicleType
Truck         64
Motorcycle    48
Bus           45
Car           43
Name: count, dtype: int64

Cleaned VehicleType values:
VehicleType_Cleaned
Truck         64
Motorcycle    48
Bus           45
Car           43
Name: count, dtype: int64
  VehicleType VehicleType_Cleaned  VehicleType_Encoded
0         Car                 Car                    1
1         Car                 Car                    1
2         Car                 Car                    1
3         Bus                 Bus                    2
4         Bus                 Bus                    2
5  Motorcycle          Motorcycle                    4
6  Motorcycle          Motorcycle                    4
7         Car                 Car                    1
8       Truck               Truck                    3
9       Truck               Truck                    3


##### 4. Owner ID

In [107]:
#1) Checking for missing OwnerIDs
missing_owners = df_vehicles['OwnerID'].isnull().sum()
print(f"Missing OwnerID values: {missing_owners}")

#2) Getting all valid DriverIDs
valid_driver_ids = set(df_drivers['DriverID'])

#3) Checking which OwnerIDs in Vehicles are not found in Drivers
df_vehicles['OwnerID_Invalid'] = ~df_vehicles['OwnerID'].isin(valid_driver_ids)
invalid_owner_rows = df_vehicles[df_vehicles['OwnerID_Invalid']]

#4) Printing the invalid OwnerIDs
print("\nOwnerIDs with no matching DriverID:")
print(invalid_owner_rows[['VehicleID', 'OwnerID']])

#5) Fix invalid OwnerIDs by assigning a default or dropping them
# Example: Replace with -1 or 'Unknown'
df_vehicles['OwnerID_Cleaned'] = df_vehicles.apply(
    lambda row: row['OwnerID'] if not row['OwnerID_Invalid'] else -1,
    axis=1
)

### Note: Thus, referential integrity is being maintained in the dataset

Missing OwnerID values: 0

OwnerIDs with no matching DriverID:
Empty DataFrame
Columns: [VehicleID, OwnerID]
Index: []


##### 5. DriverID

In [110]:
#1) Check for missing values
missing_driver_ids = df_drivers['DriverID'].isnull().sum()
print(f"Missing DriverID values: {missing_driver_ids}")

#2) Check for duplicates
duplicate_driver_ids = df_drivers['DriverID'].duplicated().sum()
print(f"Duplicate DriverID values: {duplicate_driver_ids}")

#3) Ensure the data type is integer
if not pd.api.types.is_integer_dtype(df_drivers['DriverID']):
    df_drivers['DriverID'] = pd.to_numeric(df_drivers['DriverID'], errors='coerce').astype('Int64')
    print("DriverID converted to integer type.")

#4) Drop rows with missing or duplicate IDs (optional)
df_cleaned_drivers = df_drivers.dropna(subset=['DriverID'])
df_cleaned_drivers = df_cleaned_drivers[~df_cleaned_drivers['DriverID'].duplicated()]

#5) Final confirmation
print("Are DriverIDs unique after cleaning?:", df_cleaned_drivers['DriverID'].is_unique)


Missing DriverID values: 0
Duplicate DriverID values: 0
Are DriverIDs unique after cleaning?: True


##### 6. Name

In [113]:
#1) Checking for missing values
missing_names = df_drivers['Name'].isnull().sum()
print(f"Missing Name values: {missing_names}")

#2) Function to clean and title-case names
def clean_name(name):
    if pd.isnull(name):
        return "Unknown"
    return str(name).strip().title()  # Removes leading/trailing spaces & capitalizes each word

#3) Applying the function
df_drivers['Name_Cleaned'] = df_drivers['Name'].apply(clean_name)

#5) Show some before-and-after examples
print(df_drivers[['Name', 'Name_Cleaned']].head(10))

Missing Name values: 0
              Name     Name_Cleaned
0   Muhammad Ahmed   Muhammad Ahmed
1       Ali Hassan       Ali Hassan
2  Kamran Siddiqui  Kamran Siddiqui
3      Bilal Iqbal      Bilal Iqbal
4      Usman Malik      Usman Malik
5     Farhan Ahmed     Farhan Ahmed
6       Imran Raza       Imran Raza
7       Asif Iqbal       Asif Iqbal
8    Zubair Sheikh    Zubair Sheikh
9       Adnan Khan       Adnan Khan


##### 7. Age

In [116]:
#1) Missing values in the Age col
missing_age = df_drivers['Age'].isnull().sum()
print(f"Missing Age values: {missing_age}")

#2) Invalid ages (negative values or unrealistic ones, like > 100)
invalid_ages = df_drivers[(df_drivers['Age'] < 18) | (df_drivers['Age'] > 100)]
print(f"Invalid Age values (less than 18 or more than 100):\n{invalid_ages[['DriverID', 'Age']]}")

#3) Now replacing invalid ages w/ the median or a default value
median_age = df_drivers['Age'].median()
df_drivers['Age_Cleaned'] = df_drivers['Age'].apply(
    lambda x: median_age if x < 18 or x > 100 else x
)

#4) Verifying it now
print("\nCleaned Age values:\n", df_drivers[['DriverID', 'Age', 'Age_Cleaned']].head(10))

#5) (Optional) Visualize or return the cleaned data for review
# df_drivers.to_excel("Cleaned_Drivers_Age.xlsx", index=False)  # Uncomment to save cleaned data


Missing Age values: 0
Invalid Age values (less than 18 or more than 100):
Empty DataFrame
Columns: [DriverID, Age]
Index: []

Cleaned Age values:
    DriverID  Age  Age_Cleaned
0         1   56           56
1         2   24           24
2         3   25           25
3         4   42           42
4         5   62           62
5         6   60           60
6         7   58           58
7         8   59           59
8         9   26           26
9        10   56           56


##### 8. License number

In [119]:
#1) Missing LicenseNumber values
missing_license_numbers = df_drivers['LicenseNumber'].isnull().sum()
print(f"Missing LicenseNumber values: {missing_license_numbers}")

#2) duplicate LicenseNumber values
duplicate_license_numbers = df_drivers['LicenseNumber'].duplicated().sum()
print(f"Duplicate LicenseNumber values: {duplicate_license_numbers}")

#3) List duplicate LicenseNumbers & their associated DriverIDs
duplicate_rows = df_drivers[df_drivers['LicenseNumber'].duplicated()]
print("\nDuplicate LicenseNumber rows:")
print(duplicate_rows[['DriverID', 'LicenseNumber']])

#4) Removing duplicates or handle them
df_drivers_cleaned = df_drivers.drop_duplicates(subset=['LicenseNumber'], keep='first')

#5) Confirming no more duplicates
duplicate_license_numbers_after_clean = df_drivers_cleaned['LicenseNumber'].duplicated().sum()
print(f"Duplicate LicenseNumber values after cleaning: {duplicate_license_numbers_after_clean}")



Missing LicenseNumber values: 0
Duplicate LicenseNumber values: 0

Duplicate LicenseNumber rows:
Empty DataFrame
Columns: [DriverID, LicenseNumber]
Index: []
Duplicate LicenseNumber values after cleaning: 0


##### 9. Experience Years

In [122]:
#1)Missing values in the ExperienceYears and Age columns
missing_experience = df_drivers['ExperienceYears'].isnull().sum()
print(f"Missing ExperienceYears values: {missing_experience}")
missing_age = df_drivers['Age'].isnull().sum()
print(f"Missing Age values: {missing_age}")

#2) ExperienceYears is greater than Age (invalid cases)
invalid_experience = df_drivers[df_drivers['ExperienceYears'] > df_drivers['Age']]
print(f"\nInvalid ExperienceYears (greater than Age):\n{invalid_experience[['DriverID', 'Age', 'ExperienceYears']]}")

#3) Replacing invalid ExperienceYears with Age (or handle it in another way)
df_drivers['ExperienceYears_Cleaned'] = df_drivers.apply(
    lambda row: row['Age'] if row['ExperienceYears'] > row['Age'] else row['ExperienceYears'],
    axis=1
)

#5) Confirm the cleaning process
print("\nCleaned ExperienceYears values (ExperienceYears ≤ Age):")
print(df_drivers[['DriverID', 'Age', 'ExperienceYears', 'ExperienceYears_Cleaned']].head(10))

Missing ExperienceYears values: 0
Missing Age values: 0

Invalid ExperienceYears (greater than Age):
     DriverID  Age  ExperienceYears
2           3   25               42
3           4   42               43
11         12   30               40
19         20   23               34
21         22   21               30
37         38   33               47
64         65   20               24
84         85   27               44
88         89   29               42
94         95   20               38
98         99   21               28
99        100   34               40
103       104   33               46
110       111   23               38
114       115   19               39
117       118   18               29
129       130   26               38
140       141   29               35
162       163   33               42
165       166   45               49
170       171   28               39
171       172   30               50
177       178   35               37
181       182   18               32

##### 10. Violation ID

In [129]:
# 1) Checking for missing ViolationID values
missing_violation_ids = df_speed_violations['ViolationID'].isnull().sum()
print(f"Missing ViolationID: {missing_violation_ids}")

# 2) Checking for duplicates
duplicate_violation_ids = df_speed_violations['ViolationID'].duplicated().sum()
print(f"Duplicate ViolationID: {duplicate_violation_ids}")

# 3) Showing duplicate rows
duplicate_rows = df_speed_violations[df_speed_violations['ViolationID'].duplicated()]
print("\nDuplicate rows:")
print(duplicate_rows[['ViolationID', 'VehicleID']])

# 4) Dropping duplicates and keeping the first
df_speed_violations_cleaned = df_speed_violations.drop_duplicates(subset=['ViolationID'], keep='first')

# 5) Final check for duplicates
final_duplicate_check = df_speed_violations_cleaned['ViolationID'].duplicated().sum()
print(f"Final duplicate check: {final_duplicate_check}")


Missing ViolationID: 0
Duplicate ViolationID: 0

Duplicate rows:
Empty DataFrame
Columns: [ViolationID, VehicleID]
Index: []
Final duplicate check: 0


##### 11. Vehicle ID [refrential integrity check with Speed Violations]

In [137]:
# 1) Checking if all VehicleIDs in SpeedViolations exist in Vehicles
invalid_vehicle_ids = df_speed_violations[~df_speed_violations['VehicleID'].isin(df_vehicles['VehicleID'])]
print(f"Invalid VehicleIDs (not in Vehicles table):\n{invalid_vehicle_ids[['ViolationID', 'VehicleID']]}")

# 2) Handle missing or invalid VehicleIDs (optional)
# Replace invalid VehicleIDs with a placeholder (e.g., -1) or flag them for review
df_speed_violations['VehicleID_Cleaned'] = df_speed_violations['VehicleID'].apply(
    lambda x: x if x in df_vehicles['VehicleID'].values else -1
)

# 3) Final check to ensure all VehicleIDs are valid
final_invalid_check = df_speed_violations[df_speed_violations['VehicleID_Cleaned'] == -1]
print(f"Rows with invalid VehicleID after cleaning: {final_invalid_check[['ViolationID', 'VehicleID_Cleaned']].head(10)}")


Invalid VehicleIDs (not in Vehicles table):
Empty DataFrame
Columns: [ViolationID, VehicleID]
Index: []
Rows with invalid VehicleID after cleaning: Empty DataFrame
Columns: [ViolationID, VehicleID_Cleaned]
Index: []


##### 12. Speed Recorded

In [140]:
# 1) Checking for negative speed values
negative_speeds = df_speed_violations[df_speed_violations['SpeedRecorded'] < 0]
print(f"Negative speed values:\n{negative_speeds[['ViolationID', 'SpeedRecorded']]}")

# 2) Replace negative speeds with the median or a default value (e.g., 0)
median_speed = df_speed_violations['SpeedRecorded'].median()
df_speed_violations['SpeedRecorded_Cleaned'] = df_speed_violations['SpeedRecorded'].apply(
    lambda x: x if x >= 0 else median_speed
)

# 3) Optional: Check for outliers (e.g., speeds > 200) and handle them
outliers = df_speed_violations[df_speed_violations['SpeedRecorded'] > 200]
print(f"Speed outliers (greater than 200):\n{outliers[['ViolationID', 'SpeedRecorded']]}")

# 4) Replace outliers with the upper limit or a predefined value (e.g., 200)
df_speed_violations['SpeedRecorded_Cleaned'] = df_speed_violations['SpeedRecorded_Cleaned'].apply(
    lambda x: 200 if x > 200 else x
)

# 5) Final check for cleaned speeds
print(f"Cleaned speed values:\n{df_speed_violations[['ViolationID', 'SpeedRecorded', 'SpeedRecorded_Cleaned']].head(10)}")


Negative speed values:
Empty DataFrame
Columns: [ViolationID, SpeedRecorded]
Index: []
Speed outliers (greater than 200):
Empty DataFrame
Columns: [ViolationID, SpeedRecorded]
Index: []
Cleaned speed values:
   ViolationID  SpeedRecorded  SpeedRecorded_Cleaned
0            1            140                    140
1            2            105                    105
2            3             92                     92
3            4            120                    120
4            5            113                    113
5            6            134                    134
6            7            124                    124
7            8            103                    103
8            9             65                     65
9           10            107                    107


##### 13. Speed Limit

In [145]:
# 1) Checking if SpeedLimit values are within a reasonable range (e.g., 10 to 150 km/h)
invalid_speed_limits = df_speed_violations[(df_speed_violations['SpeedLimit'] < 10) | (df_speed_violations['SpeedLimit'] > 150)]
print(f"Invalid SpeedLimit values (outside of 10-150 range):\n{invalid_speed_limits[['ViolationID', 'SpeedLimit']]}")

# 2) Handle invalid SpeedLimit values by replacing them with a default (e.g., 50)
df_speed_violations['SpeedLimit_Cleaned'] = df_speed_violations['SpeedLimit'].apply(
    lambda x: 50 if x < 10 or x > 150 else x
)

# 3) Optional: Check for unusually high SpeedLimit values (e.g., above 130 km/h in residential zones)
high_speed_limits = df_speed_violations[df_speed_violations['SpeedLimit'] > 130]
print(f"High SpeedLimit values (above 130):\n{high_speed_limits[['ViolationID', 'SpeedLimit']]}")

# 4) Handle high SpeedLimit values by capping them at 130 km/h (or a suitable limit)
df_speed_violations['SpeedLimit_Cleaned'] = df_speed_violations['SpeedLimit_Cleaned'].apply(
    lambda x: 130 if x > 130 else x
)

# 5) Final check for cleaned SpeedLimit values
print(f"Cleaned SpeedLimit values:\n{df_speed_violations[['ViolationID', 'SpeedLimit', 'SpeedLimit_Cleaned']].head(10)}")


Invalid SpeedLimit values (outside of 10-150 range):
Empty DataFrame
Columns: [ViolationID, SpeedLimit]
Index: []
High SpeedLimit values (above 130):
Empty DataFrame
Columns: [ViolationID, SpeedLimit]
Index: []
Cleaned SpeedLimit values:
   ViolationID  SpeedLimit  SpeedLimit_Cleaned
0            1          80                  80
1            2          80                  80
2            3          70                  70
3            4          70                  70
4            5          70                  70
5            6          70                  70
6            7          90                  90
7            8          50                  50
8            9          60                  60
9           10          80                  80


##### 14. Timestamp

In [150]:
# 1) Checking if the Timestamp is in a valid datetime format
invalid_timestamp = pd.to_datetime(df_speed_violations['Timestamp'], errors='coerce')
invalid_timestamps = df_speed_violations[invalid_timestamp.isna()]
print(f"Invalid Timestamp values:\n{invalid_timestamps[['ViolationID', 'Timestamp']]}")

# 2) If invalid, replace with the current timestamp
df_speed_violations['Timestamp_Cleaned'] = df_speed_violations['Timestamp'].apply(
    lambda x: pd.Timestamp('today') if pd.to_datetime(x, errors='coerce') is pd.NaT else x
)

# 3) Fixing the range check - 10 years ago from today
start_date = pd.Timestamp.today() - pd.DateOffset(years=10)  # Correct way to get 10 years ago
end_date = pd.Timestamp.today()

# Checking for timestamps outside the valid range
out_of_range_timestamps = df_speed_violations[
    (pd.to_datetime(df_speed_violations['Timestamp']) < start_date) |
    (pd.to_datetime(df_speed_violations['Timestamp']) > end_date)
]
print(f"Timestamps out of range:\n{out_of_range_timestamps[['ViolationID', 'Timestamp']]}")

# 4) If out of range, replace with the current date
df_speed_violations['Timestamp_Cleaned'] = df_speed_violations['Timestamp_Cleaned'].apply(
    lambda x: pd.Timestamp('today') if (pd.to_datetime(x) < start_date or pd.to_datetime(x) > end_date) else x
)

# 5) Final check for cleaned Timestamps
print(f"Cleaned Timestamp values:\n{df_speed_violations[['ViolationID', 'Timestamp', 'Timestamp_Cleaned']].head(10)}")


Invalid Timestamp values:
Empty DataFrame
Columns: [ViolationID, Timestamp]
Index: []
Timestamps out of range:
Empty DataFrame
Columns: [ViolationID, Timestamp]
Index: []
Cleaned Timestamp values:
   ViolationID           Timestamp   Timestamp_Cleaned
0            1 2025-02-12 16:54:07 2025-02-12 16:54:07
1            2 2025-02-23 02:07:01 2025-02-23 02:07:01
2            3 2025-03-03 09:24:10 2025-03-03 09:24:10
3            4 2025-02-03 03:45:09 2025-02-03 03:45:09
4            5 2025-02-14 06:10:16 2025-02-14 06:10:16
5            6 2025-01-06 10:42:22 2025-01-06 10:42:22
6            7 2025-01-24 00:45:49 2025-01-24 00:45:49
7            8 2025-01-04 00:30:56 2025-01-04 00:30:56
8            9 2025-02-24 09:56:41 2025-02-24 09:56:41
9           10 2025-02-19 01:43:15 2025-02-19 01:43:15


##### 15. Accident ID : Ensure uniqueness 

In [156]:
# 1) Loading the Accidents sheet
try:
    df_accidents
except NameError:
    df_accidents = pd.read_excel("Traffic_updated.xlsx", sheet_name='Accidents', engine='openpyxl')

# 2) Check how many duplicate AccidentIDs we have
duplicate_accident_ids = df_accidents['AccidentID'].duplicated().sum()
print(f"Duplicate AccidentID values: {duplicate_accident_ids}")

# 3) Show rows with duplicate AccidentIDs so we can see what's wrong
duplicate_rows = df_accidents[df_accidents['AccidentID'].duplicated()]
print("\nDuplicate AccidentID rows:")
print(duplicate_rows[['AccidentID', 'Location']])

# 4) Drop duplicates by keeping the first occurrence of each AccidentID
df_accidents_cleaned = df_accidents.drop_duplicates(subset=['AccidentID'], keep='first')

# 5) Do a final check to confirm no duplicates remain
final_duplicate_check = df_accidents_cleaned['AccidentID'].duplicated().sum()
print(f"Duplicate AccidentID values after cleaning: {final_duplicate_check}")


Duplicate AccidentID values: 0

Duplicate AccidentID rows:
Empty DataFrame
Columns: [AccidentID, Location]
Index: []
Duplicate AccidentID values after cleaning: 0


##### 16. Location	Accident:	Geocode or standardize							

In [159]:
# 1) Load the Accidents sheet if not already loaded
try:
    df_accidents
except NameError:
    df_accidents = pd.read_excel("Traffic_updated.xlsx", sheet_name='Accidents', engine='openpyxl')

# 2) Check for any missing values in the 'Location' column
missing_locations = df_accidents['Location'].isnull().sum()
print("Missing Location values:", missing_locations)

# 3) Define a function to standardize location names (trim spaces, title case, etc.)
def clean_location(loc):
    if pd.isnull(loc):
        return "Unknown"
    # Remove extra spaces and convert to title case
    return " ".join(str(loc).strip().title().split())

# 4) Apply the function to create a new column with cleaned location names
df_accidents['Location_Cleaned'] = df_accidents['Location'].apply(clean_location)

# 5) Show a sample of the original and cleaned location values for verification
print(df_accidents[['Location', 'Location_Cleaned']].head(10))


Missing Location values: 0
                Location       Location_Cleaned
0  Shahrah-e-Bandar Road  Shahrah-E-Bandar Road
1      Shahrah-e-Gulberg      Shahrah-E-Gulberg
2        Miran Shah Road        Miran Shah Road
3        Shahrah-e-PECHS        Shahrah-E-Pechs
4    Sharae Quaid-e-Azam    Sharae Quaid-E-Azam
5       Shahrah-e-Manora       Shahrah-E-Manora
6    Shahrah-e-Saeedabad    Shahrah-E-Saeedabad
7   Shahrah-e-Ibn-e-Sina   Shahrah-E-Ibn-E-Sina
8  Shahrah-e-Bahadurabad  Shahrah-E-Bahadurabad
9   Khayaban-e-Shamsheer   Khayaban-E-Shamsheer


##### 17. SeverityAccidents: Standardize categories							

In [162]:
# 1) Load the Accidents sheet (if not already loaded)
try:
    df_accidents
except NameError:
    df_accidents = pd.read_excel("Traffic_updated.xlsx", sheet_name='Accidents', engine='openpyxl')

# 2) Check how many entries are missing in the Severity field
missing_severity = df_accidents['Severity'].isnull().sum()
print("Missing Severity values:", missing_severity)

# 3) Define a function to standardize severity categories
def clean_severity(sev):
    if pd.isnull(sev):
        return "Unknown"
    # Trim and convert to lower case for consistency
    sev = str(sev).strip().lower()
    if sev in ["minor", "min", "low"]:
        return "Minor"
    elif sev in ["severe", "major", "high"]:
        return "Severe"
    else:
        return "Other"

# 4) Apply the function to create a new column with cleaned severity values
df_accidents['Severity_Cleaned'] = df_accidents['Severity'].apply(clean_severity)

# 5) Show a sample of original and cleaned Severity values
print(df_accidents[['Severity', 'Severity_Cleaned']].head(10))


Missing Severity values: 0
   Severity Severity_Cleaned
0     Minor            Minor
1     Minor            Minor
2    Severe           Severe
3     Minor            Minor
4    Severe           Severe
5     Minor            Minor
6  Moderate            Other
7    Severe           Severe
8  Moderate            Other
9     Minor            Minor


##### 18. VehiclesInvolved: Check for negatives 							

In [181]:
# 1) Loading the Accidents sheet now
try:
    df_accidents
except NameError:
    df_accidents = pd.read_excel("Traffic_updated.xlsx", sheet_name='Accidents', engine='openpyxl')

# 2) Checking for negative values in VehiclesInvolved now
negatives = df_accidents[df_accidents['VehiclesInvolved'] < 0]
print("Negatives in VehiclesInvolved:", negatives[['AccidentID', 'VehiclesInvolved']])

# 3) Replacing negative values with 1 now
df_accidents['VehiclesInvolved_Cleaned'] = df_accidents['VehiclesInvolved'].apply(lambda x: 1 if x < 0 else x)

# 4) Final check now to ensure no negatives remain
final_negatives = df_accidents[df_accidents['VehiclesInvolved_Cleaned'] < 0]
print("Final check (should be empty):", final_negatives[['AccidentID', 'VehiclesInvolved_Cleaned']])


Negatives in VehiclesInvolved: Empty DataFrame
Columns: [AccidentID, VehiclesInvolved]
Index: []
Final check (should be empty): Empty DataFrame
Columns: [AccidentID, VehiclesInvolved_Cleaned]
Index: []


##### 19. ReportedAt: Ensure valid date-time							 							

In [183]:
# 1) Loading the Accidents sheet now
try:
    df_accidents
except NameError:
    df_accidents = pd.read_excel("Traffic_updated.xlsx", sheet_name='Accidents', engine='openpyxl')

# 2) Checking ReportedAt for valid date-time now
df_accidents['ReportedAt_Converted'] = pd.to_datetime(df_accidents['ReportedAt'], errors='coerce')
invalid_reported_at = df_accidents[df_accidents['ReportedAt_Converted'].isna()]
print("Invalid ReportedAt entries:", invalid_reported_at[['AccidentID', 'ReportedAt']])

# 3) Handling invalid ReportedAt values now by replacing with today's date
df_accidents['ReportedAt_Cleaned'] = df_accidents['ReportedAt_Converted'].apply(
    lambda x: pd.Timestamp('today') if pd.isna(x) else x
)

# 4) Final check now to ensure ReportedAt is valid
print("Cleaned ReportedAt sample:")
print(df_accidents[['AccidentID', 'ReportedAt', 'ReportedAt_Cleaned']].head(10))


Invalid ReportedAt entries: Empty DataFrame
Columns: [AccidentID, ReportedAt]
Index: []
Cleaned ReportedAt sample:
   AccidentID          ReportedAt  ReportedAt_Cleaned
0           1 2025-02-20 15:15:34 2025-02-20 15:15:34
1           2 2025-01-07 13:14:37 2025-01-07 13:14:37
2           3 2025-02-03 15:03:38 2025-02-03 15:03:38
3           4 2025-02-19 01:05:37 2025-02-19 01:05:37
4           5 2025-02-15 15:15:27 2025-02-15 15:15:27
5           6 2025-01-06 20:48:02 2025-01-06 20:48:02
6           7 2025-02-01 01:52:06 2025-02-01 01:52:06
7           8 2025-02-01 17:34:58 2025-02-01 17:34:58
8           9 2025-01-13 14:28:48 2025-01-13 14:28:48
9          10 2025-02-24 02:57:45 2025-02-24 02:57:45


##### 20. WeatherID: Ensure data integrity							

In [185]:
# 1) Loading the WeatherData sheet now
try:
    df_weather
except NameError:
    df_weather = pd.read_excel("Traffic_updated.xlsx", sheet_name='WeatherData', engine='openpyxl')

# 2) Checking for missing WeatherID values now
missing_weather_id = df_weather['WeatherID'].isnull().sum()
print("Missing WeatherID:", missing_weather_id)

# 3) Checking for duplicate WeatherID values now
duplicate_weather_id = df_weather['WeatherID'].duplicated().sum()
print("Duplicate WeatherID count:", duplicate_weather_id)

# 4) Showing duplicate WeatherID rows now (if any)
duplicate_rows = df_weather[df_weather['WeatherID'].duplicated()]
print("Duplicate WeatherID rows:")
print(duplicate_rows[['WeatherID']])

# 5) Removing duplicates now by keeping the first occurrence
df_weather_cleaned = df_weather.drop_duplicates(subset=['WeatherID'], keep='first')

# 6) Final check now to confirm WeatherID is unique
final_duplicates = df_weather_cleaned['WeatherID'].duplicated().sum()
print("Final duplicate count in WeatherID:", final_duplicates)


Missing WeatherID: 0
Duplicate WeatherID count: 0
Duplicate WeatherID rows:
Empty DataFrame
Columns: [WeatherID]
Index: []
Final duplicate count in WeatherID: 0


##### 21. Temperature_C: Check for outliers


In [187]:
# 1) Loading the WeatherData sheet now (if not already loaded)
try:
    df_weather
except NameError:
    df_weather = pd.read_excel("Traffic_updated.xlsx", sheet_name='WeatherData', engine='openpyxl')

# 2) Calculate Q1, Q3, and IQR for Temperature_C
Q1 = df_weather['Temperature_C'].quantile(0.25)
Q3 = df_weather['Temperature_C'].quantile(0.75)
IQR = Q3 - Q1
print("Q1:", Q1, "Q3:", Q3, "IQR:", IQR)

# 3) Check for outliers: values below Q1 - 1.5*IQR or above Q3 + 1.5*IQR
outliers_temp = df_weather[
    (df_weather['Temperature_C'] < Q1 - 1.5 * IQR) | 
    (df_weather['Temperature_C'] > Q3 + 1.5 * IQR)
]
print("Outliers in Temperature_C:")
print(outliers_temp[['WeatherID', 'Temperature_C']])


Q1: 14.925 Q3: 30.525 IQR: 15.599999999999998
Outliers in Temperature_C:
Empty DataFrame
Columns: [WeatherID, Temperature_C]
Index: []


##### 22. Humidity_Percent: Ensure 0–100 range


In [189]:
# 1) Loading the WeatherData sheet now (if not already loaded)
try:
    df_weather
except NameError:
    df_weather = pd.read_excel("Traffic_updated.xlsx", sheet_name='WeatherData', engine='openpyxl')

# 2) Checking for Humidity_Percent values that are not between 0 and 100
out_of_range_humidity = df_weather[(df_weather['Humidity_Percent'] < 0) | (df_weather['Humidity_Percent'] > 100)]
print("Out-of-range Humidity_Percent values:")
print(out_of_range_humidity[['WeatherID', 'Humidity_Percent']])

# 3) Replacing out-of-range values by clipping them to the 0-100 range
df_weather['Humidity_Percent_Cleaned'] = df_weather['Humidity_Percent'].clip(lower=0, upper=100)

# 4) Final check to compare original vs cleaned values
print("Sample of Humidity_Percent vs Cleaned values:")
print(df_weather[['WeatherID', 'Humidity_Percent', 'Humidity_Percent_Cleaned']].head(10))


Out-of-range Humidity_Percent values:
Empty DataFrame
Columns: [WeatherID, Humidity_Percent]
Index: []
Sample of Humidity_Percent vs Cleaned values:
   WeatherID  Humidity_Percent  Humidity_Percent_Cleaned
0          1                40                        40
1          2                48                        48
2          3                59                        59
3          4                41                        41
4          5                27                        27
5          6                57                        57
6          7                31                        31
7          8                56                        56
8          9                82                        82
9         10                90                        90


##### 23. Condition: Standardize values


In [191]:
# 1) Loading the WeatherData sheet now (if not already loaded)
try:
    df_weather
except NameError:
    df_weather = pd.read_excel("Traffic_updated.xlsx", sheet_name='WeatherData', engine='openpyxl')

# 2) Checking for missing values in Condition
missing_condition = df_weather['Condition'].isnull().sum()
print("Missing Condition values:", missing_condition)

# 3) Define a function to standardize weather condition values
def clean_condition(cond):
    if pd.isnull(cond):
        return "Unknown"
    # Clean up string: trim, lower then title-case for consistency
    cond = str(cond).strip().lower()
    # Optionally map similar words to a standard value
    if "clear" in cond or "sunny" in cond:
        return "Sunny"
    elif "cloud" in cond:
        return "Cloudy"
    elif "rain" in cond:
        return "Rainy"
    elif "snow" in cond:
        return "Snowy"
    elif "fog" in cond:
        return "Foggy"
    else:
        return cond.title()

# 4) Apply the cleaning function to create a new standardized column
df_weather['Condition_Cleaned'] = df_weather['Condition'].apply(clean_condition)

# 5) Final check: show a sample of original vs. cleaned Condition values
print(df_weather[['WeatherID', 'Condition', 'Condition_Cleaned']].head(10))


Missing Condition values: 0
   WeatherID Condition Condition_Cleaned
0          1    Cloudy            Cloudy
1          2    Cloudy            Cloudy
2          3    Cloudy            Cloudy
3          4     Sunny             Sunny
4          5    Cloudy            Cloudy
5          6     Sunny             Sunny
6          7    Stormy            Stormy
7          8     Rainy             Rainy
8          9    Cloudy            Cloudy
9         10     Rainy             Rainy


##### 24. Timestamp: Validate datetime format


In [193]:
# 1) Loading the WeatherData sheet now (if not already loaded)
try:
    df_weather
except NameError:
    df_weather = pd.read_excel("Traffic_updated.xlsx", sheet_name='WeatherData', engine='openpyxl')

# 2) Converting the Timestamp column to datetime to validate its format
df_weather['Timestamp_Converted'] = pd.to_datetime(df_weather['Timestamp'], errors='coerce')

# 3) Checking for any invalid timestamps (those that couldn't be converted)
invalid_ts = df_weather[df_weather['Timestamp_Converted'].isna()]
print("Invalid Timestamp entries:")
print(invalid_ts[['WeatherID', 'Timestamp']])

# 4) Replacing invalid timestamps with today's date as a fallback
df_weather['Timestamp_Cleaned'] = df_weather['Timestamp_Converted'].fillna(pd.Timestamp('today'))

# 5) Final check: show a sample of original vs. cleaned Timestamps
print("Sample of cleaned Timestamps:")
print(df_weather[['WeatherID', 'Timestamp', 'Timestamp_Cleaned']].head(10))


Invalid Timestamp entries:
Empty DataFrame
Columns: [WeatherID, Timestamp]
Index: []
Sample of cleaned Timestamps:
   WeatherID           Timestamp   Timestamp_Cleaned
0          1 2025-01-08 05:19:23 2025-01-08 05:19:23
1          2 2025-02-22 11:50:40 2025-02-22 11:50:40
2          3 2025-02-20 05:23:27 2025-02-20 05:23:27
3          4 2025-01-09 09:43:52 2025-01-09 09:43:52
4          5 2025-01-29 17:16:29 2025-01-29 17:16:29
5          6 2025-02-15 12:51:23 2025-02-15 12:51:23
6          7 2025-02-13 23:39:08 2025-02-13 23:39:08
7          8 2025-02-13 06:54:31 2025-02-13 06:54:31
8          9 2025-01-28 22:01:41 2025-01-28 22:01:41
9         10 2025-01-03 13:59:53 2025-01-03 13:59:53


##### 25. IncidentID: Ensure uniqueness


In [195]:
# 1) Loading the TrafficIncidents sheet now
try:
    df_incidents
except NameError:
    df_incidents = pd.read_excel("Traffic_updated.xlsx", sheet_name='TrafficIncidents', engine='openpyxl')

# 2) Checking for missing IncidentID values now
missing_incident_ids = df_incidents['IncidentID'].isnull().sum()
print("Missing IncidentID values:", missing_incident_ids)

# 3) Checking for duplicate IncidentID values now
duplicate_incident_ids = df_incidents['IncidentID'].duplicated().sum()
print("Duplicate IncidentID count:", duplicate_incident_ids)

# 4) Showing duplicate IncidentID rows now
duplicate_rows = df_incidents[df_incidents['IncidentID'].duplicated()]
print("Duplicate IncidentID rows:")
print(duplicate_rows[['IncidentID', 'Type']])

# 5) Dropping duplicates now and keeping the first occurrence
df_incidents_cleaned = df_incidents.drop_duplicates(subset=['IncidentID'], keep='first')

# 6) Final check now to confirm IncidentID is unique
final_duplicate_check = df_incidents_cleaned['IncidentID'].duplicated().sum()
print("Final duplicate count for IncidentID:", final_duplicate_check)


Missing IncidentID values: 0
Duplicate IncidentID count: 0
Duplicate IncidentID rows:
Empty DataFrame
Columns: [IncidentID, Type]
Index: []
Final duplicate count for IncidentID: 0


##### 26. Type: Group and standardize

In [197]:
# 1) Loading the TrafficIncidents sheet now (if not already loaded)
try:
    df_incidents
except NameError:
    df_incidents = pd.read_excel("Traffic_updated.xlsx", sheet_name='TrafficIncidents', engine='openpyxl')

# 2) Checking for missing incident types now
missing_types = df_incidents['Type'].isnull().sum()
print("Missing incident types:", missing_types)

# 3) Defining a function to group and standardize incident types
def clean_incident_type(itype):
    if pd.isnull(itype):
        return "Unknown"
    itype = str(itype).strip().lower()
    if "breakdown" in itype:
        return "Vehicle Breakdown"
    elif "road rage" in itype:
        return "Road Rage"
    elif "accident" in itype:
        return "Accident"
    else:
        return "Other"

# 4) Applying the function to create a new column with standardized types
df_incidents['Type_Cleaned'] = df_incidents['Type'].apply(clean_incident_type)

# 5) Final check: preview original vs. cleaned incident types
print(df_incidents[['Type', 'Type_Cleaned']].head(10))


Missing incident types: 0
                Type       Type_Cleaned
0  Vehicle Breakdown  Vehicle Breakdown
1  Vehicle Breakdown  Vehicle Breakdown
2          Road Rage          Road Rage
3  Vehicle Breakdown  Vehicle Breakdown
4          Road Rage          Road Rage
5        Hit and Run              Other
6          Road Rage          Road Rage
7  Vehicle Breakdown  Vehicle Breakdown
8        Hit and Run              Other
9  Vehicle Breakdown  Vehicle Breakdown


##### 27. Location: Standardize

In [199]:
# 1) Loading the TrafficIncidents sheet now (if not already loaded)
try:
    df_incidents
except NameError:
    df_incidents = pd.read_excel("Traffic_updated.xlsx", sheet_name='TrafficIncidents', engine='openpyxl')

# 2) Checking for missing values in 'Location'
missing_locations = df_incidents['Location'].isnull().sum()
print("Missing incident locations:", missing_locations)

# 3) Defining a function to standardize the location names
def clean_location(loc):
    if pd.isnull(loc):
        return "Unknown"
    return " ".join(str(loc).strip().title().split())

# 4) Applying the function to create a cleaned location column
df_incidents['Location_Cleaned'] = df_incidents['Location'].apply(clean_location)

# 5) Final check: Show sample of original vs. cleaned location values
print(df_incidents[['Location', 'Location_Cleaned']].head(10))


Missing incident locations: 0
                Location       Location_Cleaned
0       Shahrah-e-Faisal       Shahrah-E-Faisal
1         MA Jinnah Road         Ma Jinnah Road
2     II Chundrigar Road     Ii Chundrigar Road
3        University Road        University Road
4     Rashid Minhas Road     Rashid Minhas Road
5     Shahrah-e-Pakistan     Shahrah-E-Pakistan
6           Korangi Road           Korangi Road
7     Shahrah-e-Quaideen     Shahrah-E-Quaideen
8  Shaheed-e-Millat Road  Shaheed-E-Millat Road
9      Shahrah-e-Gulberg      Shahrah-E-Gulberg


##### 28. ReportedAt: Ensure consistent datetime format

In [210]:
# 1) Loading the TrafficIncidents sheet now (if not already loaded)
try:
    df_incidents
except NameError:
    df_incidents = pd.read_excel("Traffic_updated.xlsx", sheet_name='TrafficIncidents', engine='openpyxl')

# 2) Converting 'ReportedAt' to datetime format
df_incidents['ReportedAt_Converted'] = pd.to_datetime(df_incidents['ReportedAt'], errors='coerce')

# 3) Checking for any invalid date-time entries
invalid_reported = df_incidents[df_incidents['ReportedAt_Converted'].isna()]
print("Invalid ReportedAt entries:")
print(invalid_reported[['IncidentID', 'ReportedAt']])

# 4) Replacing invalid date-times with today's timestamp
df_incidents['ReportedAt_Cleaned'] = df_incidents['ReportedAt_Converted'].fillna(pd.Timestamp('today'))

# 5) Final check: preview original vs. cleaned ReportedAt values
print("Sample of original vs. cleaned ReportedAt:")
print(df_incidents[['ReportedAt', 'ReportedAt_Cleaned']].head(10))


Invalid ReportedAt entries:
Empty DataFrame
Columns: [IncidentID, ReportedAt]
Index: []
Sample of original vs. cleaned ReportedAt:
           ReportedAt  ReportedAt_Cleaned
0 2025-02-15 14:46:00 2025-02-15 14:46:00
1 2025-02-23 18:29:41 2025-02-23 18:29:41
2 2025-01-15 23:55:50 2025-01-15 23:55:50
3 2025-02-05 22:04:32 2025-02-05 22:04:32
4 2025-01-15 19:18:05 2025-01-15 19:18:05
5 2025-01-20 00:55:24 2025-01-20 00:55:24
6 2025-01-30 07:56:37 2025-01-30 07:56:37
7 2025-01-31 08:08:58 2025-01-31 08:08:58
8 2025-03-03 00:41:46 2025-03-03 00:41:46
9 2025-02-17 06:04:21 2025-02-17 06:04:21


##### 29. Location ID: Ensure data integrity

In [212]:
# 1) Loading the Location sheet now (if it's not already loaded)
try:
    df_location
except NameError:
    df_location = pd.read_excel("Traffic_updated.xlsx", sheet_name='Location', engine='openpyxl')

# 2) Checking for missing values in "Location ID" now
missing_location_ids = df_location['Location ID'].isnull().sum()
print(f"Missing Location ID values: {missing_location_ids}")

# 3) Checking for duplicate Location ID values now
duplicate_location_ids = df_location['Location ID'].duplicated().sum()
print(f"Duplicate Location ID values: {duplicate_location_ids}")

# 4) Showing rows with duplicate Location IDs now
duplicate_rows = df_location[df_location['Location ID'].duplicated()]
print("Duplicate rows in Location ID:")
print(duplicate_rows[['Location ID', 'Road Name']])

# 5) Removing duplicates now (keeping the first occurrence)
df_location_cleaned = df_location.drop_duplicates(subset=['Location ID'], keep='first')

# 6) Final check now to confirm Location ID is unique
final_duplicate_check = df_location_cleaned['Location ID'].duplicated().sum()
print(f"Final duplicate count in Location ID: {final_duplicate_check}")


Missing Location ID values: 0
Duplicate Location ID values: 0
Duplicate rows in Location ID:
Empty DataFrame
Columns: [Location ID, Road Name]
Index: []
Final duplicate count in Location ID: 0


##### 30. Road Name: Standardize names

In [214]:
# 1) Loading the Location sheet now (if not already loaded)
try:
    df_location
except NameError:
    df_location = pd.read_excel("Traffic_updated.xlsx", sheet_name='Location', engine='openpyxl')

# 2) Checking for missing values in "Road Name"
missing_road_name = df_location['Road Name'].isnull().sum()
print("Missing Road Name values:", missing_road_name)

# 3) Defining a function to standardize road names (trim spaces, convert to title case)
def clean_road_name(road):
    if pd.isnull(road):
        return "Unknown"
    return " ".join(str(road).strip().title().split())

# 4) Applying the function to create a new column with cleaned road names
df_location['Road Name_Cleaned'] = df_location['Road Name'].apply(clean_road_name)

# 5) Final check now: preview original vs. cleaned Road Name values
print(df_location[['Road Name', 'Road Name_Cleaned']].head(10))


Missing Road Name values: 0
               Road Name      Road Name_Cleaned
0       Shahrah-e-Faisal       Shahrah-E-Faisal
1         MA Jinnah Road         Ma Jinnah Road
2     II Chundrigar Road     Ii Chundrigar Road
3        University Road        University Road
4     Rashid Minhas Road     Rashid Minhas Road
5     Shahrah-e-Pakistan     Shahrah-E-Pakistan
6           Korangi Road           Korangi Road
7     Shahrah-e-Quaideen     Shahrah-E-Quaideen
8  Shaheed-e-Millat Road  Shaheed-E-Millat Road
9       Sunset Boulevard       Sunset Boulevard


##### 31. Area/Location: Format location info

In [216]:
# 1) Loading the Location sheet now (if not already loaded)
try:
    df_location
except NameError:
    df_location = pd.read_excel("Traffic_updated.xlsx", sheet_name='Location', engine='openpyxl')

# 2) Checking for missing values in the 'Area/Location' column
missing_area = df_location['Area/Location'].isnull().sum()
print("Missing Area/Location values:", missing_area)

# 3) Defining a function to clean and format location info (trim spaces, convert to title case)
def clean_area_location(loc):
    if pd.isnull(loc):
        return "Unknown"
    return " ".join(str(loc).strip().title().split())

# 4) Applying the function to create a new cleaned column
df_location['Area/Location_Cleaned'] = df_location['Area/Location'].apply(clean_area_location)

# 5) Final check: Preview original vs. cleaned Area/Location values
print(df_location[['Area/Location', 'Area/Location_Cleaned']].head(10))


Missing Area/Location values: 0
                          Area/Location                 Area/Location_Cleaned
0  Jinnah International Airport to PIDC  Jinnah International Airport To Pidc
1                    Saddar to Kharadar                    Saddar To Kharadar
2             Tower to Merewether Tower             Tower To Merewether Tower
3       NED University to Hassan Square       Ned University To Hassan Square
4          Gulshan to Gulistan-e-Jauhar          Gulshan To Gulistan-E-Jauhar
5            Sohrab Goth to Liaquatabad            Sohrab Goth To Liaquatabad
6        Korangi Crossing to Qayyumabad        Korangi Crossing To Qayyumabad
7                       Clifton to NIPA                       Clifton To Nipa
8              Nursery to KDA Chowrangi              Nursery To Kda Chowrangi
9                           DHA Phase 2                           Dha Phase 2


##### 32. FlowID: Ensure uniqueness

In [218]:
# 1) Loading the TrafficFlow sheet now
try:
    df_traffic_flow
except NameError:
    df_traffic_flow = pd.read_excel("Traffic_updated.xlsx", sheet_name='TrafficFlow', engine='openpyxl')

# 2) Checking for missing FlowID values now
missing_flow_ids = df_traffic_flow['FlowID'].isnull().sum()
print("Missing FlowID values:", missing_flow_ids)

# 3) Checking for duplicate FlowID values now
duplicate_flow_ids = df_traffic_flow['FlowID'].duplicated().sum()
print("Duplicate FlowID count:", duplicate_flow_ids)

# 4) Dropping duplicates now and keeping the first occurrence
df_traffic_flow_cleaned = df_traffic_flow.drop_duplicates(subset=['FlowID'], keep='first')

# 5) Final check now to ensure no duplicates remain
final_duplicate_check = df_traffic_flow_cleaned['FlowID'].duplicated().sum()
print("Final duplicate check for FlowID:", final_duplicate_check)


Missing FlowID values: 0
Duplicate FlowID count: 0
Final duplicate check for FlowID: 0


##### 33. Location ID: Referential integrity

In [220]:
# 1) Loading the TrafficFlow sheet now (if not already loaded)
try:
    df_traffic_flow
except NameError:
    df_traffic_flow = pd.read_excel("Traffic_updated.xlsx", sheet_name='TrafficFlow', engine='openpyxl')

# 2) Loading the Location sheet now (if not already loaded)
try:
    df_location
except NameError:
    df_location = pd.read_excel("Traffic_updated.xlsx", sheet_name='Location', engine='openpyxl')

# 3) Checking which Location IDs in TrafficFlow aren't in the Location sheet
invalid_location_ids = df_traffic_flow[~df_traffic_flow['Location ID'].isin(df_location['Location ID'])]
print("Invalid Location IDs in TrafficFlow (not in Location sheet):")
print(invalid_location_ids[['FlowID', 'Location ID']])

# 4) Replacing invalid Location IDs with a default value (-1)
df_traffic_flow['Location ID_Cleaned'] = df_traffic_flow['Location ID'].apply(
    lambda x: x if x in set(df_location['Location ID']) else -1
)

# 5) Final check: show rows with invalid Location IDs after cleaning
invalid_after = df_traffic_flow[df_traffic_flow['Location ID_Cleaned'] == -1]
print("Rows with invalid Location ID after cleaning:")
print(invalid_after[['FlowID', 'Location ID_Cleaned']])


Invalid Location IDs in TrafficFlow (not in Location sheet):
Empty DataFrame
Columns: [FlowID, Location ID]
Index: []
Rows with invalid Location ID after cleaning:
Empty DataFrame
Columns: [FlowID, Location ID_Cleaned]
Index: []


##### 34. VehicleCount: Detect outliers and negatives

In [222]:
# 1) Check for negative VehicleCount values now
negatives = df_traffic_flow[df_traffic_flow['VehicleCount'] < 0]
print("Negative VehicleCount values:", negatives[['FlowID', 'VehicleCount']])

# 2) Calculate Q1, Q3, and IQR for VehicleCount now
Q1 = df_traffic_flow['VehicleCount'].quantile(0.25)
Q3 = df_traffic_flow['VehicleCount'].quantile(0.75)
IQR = Q3 - Q1
print("Q1:", Q1, "Q3:", Q3, "IQR:", IQR)

# 3) Detect outliers now: values below Q1 - 1.5*IQR or above Q3 + 1.5*IQR
outliers = df_traffic_flow[(df_traffic_flow['VehicleCount'] < Q1 - 1.5 * IQR) | 
                           (df_traffic_flow['VehicleCount'] > Q3 + 1.5 * IQR)]
print("Outliers in VehicleCount:", outliers[['FlowID', 'VehicleCount']])


Negative VehicleCount values: Empty DataFrame
Columns: [FlowID, VehicleCount]
Index: []
Q1: 302.25 Q3: 788.5 IQR: 486.25
Outliers in VehicleCount: Empty DataFrame
Columns: [FlowID, VehicleCount]
Index: []


##### 35. Timestamp: Ensure valid datetime

In [224]:
# 1) Loading the TrafficFlow sheet now (if not already loaded)
try:
    df_traffic_flow
except NameError:
    df_traffic_flow = pd.read_excel("Traffic_updated.xlsx", sheet_name='TrafficFlow', engine='openpyxl')

# 2) Converting the Timestamp column to datetime
df_traffic_flow['Timestamp_Converted'] = pd.to_datetime(df_traffic_flow['Timestamp'], errors='coerce')

# 3) Checking for any invalid timestamps now (conversion failed)
invalid_ts = df_traffic_flow[df_traffic_flow['Timestamp_Converted'].isna()]
print("Rows with invalid Timestamp values:")
print(invalid_ts[['FlowID', 'Timestamp']])

# 4) Handling invalid timestamps: replace with current timestamp if needed
df_traffic_flow['Timestamp_Cleaned'] = df_traffic_flow['Timestamp_Converted'].fillna(pd.Timestamp('today'))

# 5) Final check: preview a sample of original and cleaned Timestamp values
print("Sample of original vs. cleaned Timestamps:")
print(df_traffic_flow[['FlowID', 'Timestamp', 'Timestamp_Cleaned']].head(10))


Rows with invalid Timestamp values:
Empty DataFrame
Columns: [FlowID, Timestamp]
Index: []
Sample of original vs. cleaned Timestamps:
   FlowID           Timestamp   Timestamp_Cleaned
0       1 2025-02-15 22:08:31 2025-02-15 22:08:31
1       2 2025-02-05 18:12:09 2025-02-05 18:12:09
2       3 2025-02-17 07:56:45 2025-02-17 07:56:45
3       4 2025-02-28 05:24:57 2025-02-28 05:24:57
4       5 2025-02-26 03:51:13 2025-02-26 03:51:13
5       6 2025-01-29 13:06:11 2025-01-29 13:06:11
6       7 2025-01-17 16:34:50 2025-01-17 16:34:50
7       8 2025-01-28 03:56:05 2025-01-28 03:56:05
8       9 2025-01-15 04:42:43 2025-01-15 04:42:43
9      10 2025-01-06 02:52:23 2025-01-06 02:52:23


## FACT TABLE CLEANING 

##### 1. Vehicle-id: Ensure referential integrity


##### 2. Driver-id: Ensure referential integrity


##### 3. Location-id: Standardize & create location mapping


##### 4. Incident-date: Extract from ReportedAt


##### 5. Accident-date: Extract from ReportedAt


##### 6. Incident-time (hrs): Extract time from ReportedAt


##### 7. Accident-time (hrs): Extract time from ReportedAt


##### 8. Incident-type: Standardize & encode


##### 9. Accident-severity: Standardize severity (e.g., Minor/Severe)


##### 10. Accident-vehicles-involved: Ensure non-negative


##### 11. Road-visibility: Standardize categories


##### 12. Weather-condition: Standardize categories


##### 13. Driver-age: Ensure range compliance


##### 14. Driver-experience: Fix if negative or too high


##### 15. Reason-recorded: Group similar causes; standardize